In [1]:
import polars as pl
from typing import *
import sys
sys.path.append('../src')

# General Usage

Here I demonstrate a sketch notebook in a data scientist's job.

In [2]:
import polars as pl
from dsds.utils import get_numpy
from dsds.prescreen import *
from dsds.transform import *
import dsds.fs as fs
# from dsds.fs import discrete_ig, f_classification, mrmr

In [3]:
df = pl.read_csv("../data/advertising.csv").with_columns(
    pl.lit("abc@google.com")
)
target = "Clicked on Ad"
df.head() 

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Constant,Test_Str_Constant,Test_BadColumn,Test_Binary,literal
i64,f64,i64,f64,f64,str,str,i64,str,str,i64,str,i64,i64,i64,i64,str,i64,str,str
1,68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""",0,"""Tunisia""","""3/27/2016 0:53…",0,"""A""",30,12,12,1,"""SSS""",0,"""A""","""abc@google.com…"
2,80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""",1,"""Nauru""","""4/4/2016 1:39""",0,"""B""",30,9,13,1,"""SSS""",null,"""B""","""abc@google.com…"
3,69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""",0,"""San Marino""","""3/13/2016 20:3…",0,"""A""",20,11,11,1,"""SSS""",0,"""A""","""abc@google.com…"
4,74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…",1,"""Italy""","""1/10/2016 2:31…",0,"""B""",20,12,10,1,"""SSS""",null,"""B""","""abc@google.com…"
5,68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""",0,"""Iceland""","""6/3/2016 3:36""",0,"""A""",30,11,14,1,"""SSS""",0,"""A""","""abc@google.com…"


In [4]:
describe(df)

column,count,null_count,null_pct,n_unique,unique_pct,mean,std,min,max,25%,median,75%,skew,kurtosis,dtype
str,f64,f64,f64,u32,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str
"""id""",1000.0,0.0,0.0,1000,1.0,500.5,288.819436,"""1.0""","""1000.0""",251.0,500.5,751.0,0.0,0.0,"""numeric"""
"""Daily Time Spe…",1000.0,0.0,0.0,900,0.9,65.0002,15.853615,"""32.6""","""91.43""",51.38,68.215,78.57,-0.371203,-0.371203,"""numeric"""
"""Age""",1000.0,0.0,0.0,43,0.043,36.009,8.785562,"""19.0""","""61.0""",29.0,35.0,42.0,0.478423,0.478423,"""numeric"""
"""Area Income""",1000.0,7.0,0.007,994,0.994,54980.756103,13434.996236,"""13996.5""","""79484.8""",47051.02,57009.76,65461.92,-0.648682,-0.648682,"""numeric"""
"""Daily Internet…",1000.0,11.0,0.011,957,0.957,180.024985,43.903142,"""104.78""","""269.96""",138.71,183.42,218.79,-0.04022,-0.04022,"""numeric"""
"""Ad Topic Line""",1000.0,0.0,0.0,1000,1.0,null,null,"""Adaptive 24hou…","""Visionary reci…",null,null,null,null,null,"""string"""
"""City""",1000.0,0.0,0.0,969,0.969,null,null,"""Adamsbury""","""Zacharyton""",null,null,null,null,null,"""string"""
"""Male""",1000.0,0.0,0.0,2,0.002,0.481,0.499889,"""0.0""","""1.0""",0.0,0.0,1.0,0.076055,0.076055,"""numeric"""
"""Country""",1000.0,0.0,0.0,237,0.237,null,null,"""Afghanistan""","""Zimbabwe""",null,null,null,null,null,"""string"""


In [5]:
describe_str(df, words_to_count=["A", "Visionary"])

features,null_count,min,max,mode,min_byte_len,max_byte_len,avg_byte_len,median_byte_len,avg_space_cnt,avg_digit_cnt,avg_cap_cnt,avg_lower_cnt,total_A_count,total_Visionary_count
str,i64,str,str,str,i64,i64,f64,f64,f64,f64,f64,f64,u32,u32
"""Ad Topic Line""",0,"""Visionary reci…","""Adaptive 24hou…","""Focused high-l…",17,55,33.394,33.0,2.219,0.087,1.108,29.221,60,7
"""City""",0,"""Zacharyton""","""Adamsbury""","""Williamsport""",5,23,11.887,11.0,0.471,0.0,1.471,9.945,65,0
"""Country""",0,"""Zimbabwe""","""Afghanistan""","""Czech Republic…",4,51,10.309,8.0,0.494,0.006,1.424,8.328,107,0
"""Timestamp""",0,"""7/9/2016 16:23…","""1/1/2016 15:14…","""5/20/2016 12:1…",13,15,14.273,14.0,1.0,10.273,0.0,0.0,0,0
"""One_Hot_Test""",0,"""C""","""A""","""A""",1,1,1.0,1.0,0.0,0.0,1.0,0.0,783,0
"""Test_Str_Const…",0,"""SSS""","""SSS""","""SSS""",3,3,3.0,3.0,0.0,0.0,3.0,0.0,0,0
"""Test_Binary""",0,"""B""","""A""","""A""",1,1,1.0,1.0,0.0,0.0,1.0,0.0,519,0
"""literal""",0,"""abc@google.com…","""abc@google.com…","""abc@google.com…",14,14,14.0,14.0,0.0,0.0,0.0,12.0,0,0


In [6]:
df = email_removal(df)

INFO:dsds.prescreen:The following columns are dropped because they are emails. ['literal'].
Removed a total of 1 columns.


In [7]:
df = date_removal(df)

INFO:dsds.prescreen:Date Inferral is error prone due to the huge variety of date formats. Please use with caution.
INFO:dsds.prescreen:The following columns are dropped because they are dates. ['Timestamp'].
Removed a total of 1 columns.


In [8]:
df.head()

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Constant,Test_Str_Constant,Test_BadColumn,Test_Binary
i64,f64,i64,f64,f64,str,str,i64,str,i64,str,i64,i64,i64,i64,str,i64,str
1,68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""",0,"""Tunisia""",0,"""A""",30,12,12,1,"""SSS""",0,"""A"""
2,80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""",1,"""Nauru""",0,"""B""",30,9,13,1,"""SSS""",null,"""B"""
3,69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""",0,"""San Marino""",0,"""A""",20,11,11,1,"""SSS""",0,"""A"""
4,74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…",1,"""Italy""",0,"""B""",20,12,10,1,"""SSS""",null,"""B"""
5,68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""",0,"""Iceland""",0,"""A""",30,11,14,1,"""SSS""",0,"""A"""


In [9]:
df = var_removal(df, threshold=0.5, target=target)

INFO:dsds.prescreen:The following columns are dropped because they have lower than 0.5 variance. ['Male', 'Test_Constant', 'Test_BadColumn'].
Removed a total of 3 columns.


In [10]:
df = constant_removal(df)

INFO:dsds.prescreen:The following columns are dropped because they are constants. ['Test_Str_Constant'].
Removed a total of 1 columns.


In [11]:
# This turns binary strings into 0s and 1s based on sort order. This will not transform boolean or numeric binary values.
df = binary_encode(df, exclude = [target])

In [12]:
df.head()

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary_B
i64,f64,i64,f64,f64,str,str,str,i64,str,i64,i64,i64,u8
1,68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12,12,0
2,80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9,13,1
3,69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11,11,0
4,74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12,10,1
5,68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11,14,0


In [13]:
# Second way (imo, better) to interactive with TransformationResult objects
df = impute(df, ["Area Income"], "mean")
df = impute(df, ["Daily Internet Usage", "Daily Internet Usage Band", "Area Income Band"], "median")
df 

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary_B
i64,f64,i64,f64,f64,str,str,str,i64,str,i64,f64,f64,u8
1,68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12.0,12.0,0
2,80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9.0,13.0,1
3,69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11.0,11.0,0
4,74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12.0,10.0,1
5,68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11.0,14.0,0
6,59.99,23,59761.56,226.74,"""Sharable clien…","""Jamieberg""","""Norway""",0,"""B""",20,11.0,11.0,1
7,88.91,33,54980.756103,208.36,"""Enhanced dedic…","""Brandonstad""","""Myanmar""",0,"""A""",30,10.0,11.0,0
8,66.0,48,24593.33,131.76,"""Reactive local…","""Port Jefferybu…","""Australia""",1,"""A""",40,6.0,4.0,1
9,74.53,30,68862.0,221.51,"""Configurable c…","""West Colin""","""Grenada""",0,"""A""",30,11.0,13.0,1


In [14]:
df = scale(df, ["Area Income", "Daily Internet Usage"], "standard")
df 

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary_B
i64,f64,i64,f64,f64,str,str,str,i64,str,i64,f64,f64,u8
1,68.95,35,0.511893,1.74127,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12.0,12.0,0
2,80.23,31,1.005471,0.313948,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9.0,13.0,1
3,69.47,26,0.358921,1.292598,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11.0,11.0,0
4,74.15,29,-0.01304,1.507658,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12.0,10.0,1
5,68.37,35,1.412418,1.042496,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11.0,14.0,0
6,59.99,23,0.3571,1.069064,"""Sharable clien…","""Jamieberg""","""Norway""",0,"""B""",20,11.0,11.0,1
7,88.91,33,-1.0869e-15,0.648105,"""Enhanced dedic…","""Brandonstad""","""Myanmar""",0,"""A""",30,10.0,11.0,0
8,66.0,48,-2.269777,-1.106273,"""Reactive local…","""Port Jefferybu…","""Australia""",1,"""A""",40,6.0,4.0,1
9,74.53,30,1.036854,0.94928,"""Configurable c…","""West Colin""","""Grenada""",0,"""A""",30,11.0,13.0,1


In [15]:
# Some of these are numerical columns, but we can still treat them as "discrete" columns
# and feed them into the information gain algorithm.
discrete = ["Age Band", "Country", "Area Income Band", "Daily Internet Usage Band", "Test_Binary", "One_Hot_Test"]
target = "Clicked on Ad"

In [16]:
fs.discrete_ig(df, target, discrete_cols=discrete)

ColumnNotFoundError: Test_Binary

Error originated just after this operation:
DF ["id", "Daily Time Spent on Site", "Age", "Area Income"]; PROJECT */14 COLUMNS; SELECTION: "None"

In [ ]:
# If nothing is given, automatically infers discrete columns. (See the docstring of discrete_inferral)
fs.discrete_ig(df, target)

In [ ]:
# df["Ad Topic Line"].unique() # has 1000 uniques. 
# There is no randomness at all because we can say that there are two distinct subsets that perfects differentiates 0s from 1s

In [ ]:
fs.f_classif(df, target=target)

In [ ]:
nums = get_numeric_cols(df, exclude=[target])
nums 

In [ ]:
from sklearn.feature_selection import f_classif
 
f, pv = f_classif(df.select(nums), df[target])
pl.from_records([nums, f, pv], schema=["feature", "f_value", "p_value"])

In [ ]:
from sklearn.feature_selection import f_regression

result = f_regression(df[nums].to_numpy(), df[target].to_numpy())
pl.from_records([nums, result[0], result[1]], schema=["feature", "f_value", "p_value"])

In [ ]:
df_test = pl.concat([df.clone()] * 500)
df_test.shape

In [ ]:
%%timeit 
fs.f_classif(df_test, target="Clicked on Ad", num_cols=nums)

In [ ]:
%%timeit 
f, pv = f_classif(df_test[nums].to_numpy(), df_test[target].to_numpy())
pl.from_records([nums, f, pv], schema=["feature", "f_value", "p_value"])

In [ ]:
%%timeit
result = f_regression(df_test[nums].to_numpy(), df_test[target].to_numpy())
pl.from_records([nums, result[0], result[1]], schema=["feature", "f_value", "p_value"])

In [ ]:
del df_test 

In [ ]:
# MRMR Method 
fs.mrmr(df, target, 5)

In [ ]:
df = one_hot_encode(df, cols=["One_Hot_Test"])

In [ ]:
df.head()

In [ ]:
df = ordinal_auto_encode(df, cols=["City", "Country"])

In [ ]:
df.head()

In [ ]:
final_df = remove_if_exists(df, ["Ad Topic Line", "Timestamp"]) # Timestamp was removed at the beginning
final_df.head() 

In [ ]:
np_data = get_numpy(final_df, target)
X, y, features = np_data.X, np_data.y, np_data.features

In [ ]:
X.shape 

In [ ]:
y.shape 

In [ ]:
features